In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data_paths import *

In [186]:
df = pd.read_pickle(MERGED)

In [187]:
cosmic_proteins = pd.read_csv(COSMIC_PROTEINS, sep='\t')

In [188]:
cosmic_proteins.head(1)

,Gene,Gene synonym,Ensembl,Gene description,Uniprot,Chromosome,Position,Protein class,Biological process,Molecular function,...,Cancer prognostics - Pancreatic Adenocarcinoma (TCGA),Cancer prognostics - Pancreatic Adenocarcinoma (validation),Cancer prognostics - Prostate Adenocarcinoma (TCGA),Cancer prognostics - Rectum Adenocarcinoma (TCGA),Cancer prognostics - Rectum Adenocarcinoma (validation),Cancer prognostics - Skin Cuteneous Melanoma (TCGA),Cancer prognostics - Stomach Adenocarcinoma (TCGA),Cancer prognostics - Testicular Germ Cell Tumor (TCGA),Cancer prognostics - Thyroid Carcinoma (TCGA),Cancer prognostics - Uterine Corpus Endometrial Carcinoma (TCGA)
0,ABI1,"ABI-1, E3B1, SSH3BP1",ENSG00000136754,Abl interactor 1,Q8IZP0,10,26746593-26861087,"Cancer-related genes, Disease related genes, P...",Host-virus interaction,NaN,...,unprognostic (6.17e-3),unprognostic (9.66e-2),unprognostic (8.60e-2),unprognostic (1.16e-1),unprognostic (3.28e-2),unprognostic (1.01e-1),unprognostic (3.66e-1),unprognostic (1.72e-1),unprognostic (1.03e-1),unprognostic (1.72e-1)


In [189]:
cosmic_proteins['Gene']

0        ABI1
1        ABL1
2        ABL2
3       ACKR3
4       ACSL3
        ...  
567     ZMYM2
568    ZNF331
569    ZNF384
570    ZNF521
571     ZRSR2
Name: Gene, Length: 572, dtype: object

In [190]:
arr = cosmic_proteins['Gene synonym'].apply(lambda x: np.array([y.strip() for y in str(x).split(',')]))
all_elems = np.concatenate(arr)

# Check isin for each element in the array (careful of double lists)
df['target_name'].isin(all_elems).sum(), df['name_y'].isin(all_elems).sum()

(np.int64(0), np.int64(0))

In [191]:
df['name_y'].isin(cosmic_proteins['Gene']).sum()

np.int64(20199)

In [192]:
df['target_name'].isin(cosmic_proteins['Gene']).sum()

np.int64(0)

In [193]:
cosmic_proteins['Gene'].isin(df['name_y']).sum()

np.int64(45)

In [194]:
df[df['name_y'].isin(cosmic_proteins['Gene'])].head(2)

,ki,ph,temp,ic50,ec50,kd,kon,koff,doi,target_name,...,chembl_id_DrugBank,smiles_DrugBank,inchi_key_DrugBank,chebi_id_DrugBank,bindingdb_id_DrugBank,Matched_On,name_y,general-function,specific-function,organism
8960,<NA>,7.6,22.0,840.0,<NA>,<NA>,<NA>,<NA>,10.1021/jm0201722,Cyclin-dependent kinase 4/G1/S-specific cyclin-D1,...,NaN,NaN,NaN,NaN,NaN,NaN,CCND1,Transcription factor binding,Regulatory component of the cyclin D1-CDK4 (DC...,Humans
8961,<NA>,7.6,22.0,490.0,<NA>,<NA>,<NA>,<NA>,10.1021/jm0201722,Cyclin-dependent kinase 4/G1/S-specific cyclin-D1,...,NaN,NaN,NaN,NaN,NaN,NaN,CCND1,Transcription factor binding,Regulatory component of the cyclin D1-CDK4 (DC...,Humans


In [195]:
cancer_proteins = df[df['name_y'].isin(cosmic_proteins['Gene'])]

ligands_related_to_cancer_proteins = cancer_proteins['name_x'].unique()
len(ligands_related_to_cancer_proteins), ligands_related_to_cancer_proteins

(191,
 array([nan, 'Alvocidib', 'Olomoucine', 'G9ZF61LE7G',
        "N'-(Pyrrolidino[2,1-B]Isoindolin-4-On-8-Yl)-N-(Pyridin-2-Yl)Urea",
        '1-(5-OXO-2,3,5,9B-TETRAHYDRO-1H-PYRROLO[2,1-A]ISOINDOL-9-YL)-3-(5-PYRROLIDIN-2-YL-1H-PYRAZOL-3-YL)-UREA',
        'WS821Z52LQ', 'Pteroic acid', 'K-252a',
        '6,7,12,13-tetrahydro-5H-indolo[2,3-a]pyrrolo[3,4-c]carbazol-5-one',
        'SB-218078', 'Bisindolylmaleimide I', 'Staurosporine',
        '64H5U84UFB', 'NTZ47MG2A4',
        '4-[4-(4-Methyl-2-Methylamino-Thiazol-5-Yl)-Pyrimidin-2-Ylamino]-Phenol',
        '[4-(2-Amino-4-Methyl-Thiazol-5-Yl)-Pyrimidin-2-Yl]-(3-Nitro-Phenyl)-Amine',
        "O6-CYCLOHEXYLMETHOXY-2-(4'-SULPHAMOYLANILINO) PURINE",
        'T61871RKRI', 'Wortmannin', '7-Hydroxystaurosporine', 'Pictilisib',
        'TG100-115', 'LY-294002', 'Milciclib Maleate', 'Roginolisib',
        'Tirabrutinib', 'Navtemadlin', 'Y2SR66P7VM', 'Trilaciclib',
        'Lerociclib', 'AUY922', 'Vorasidenib', 'Enasidenib', 'Abemaciclib',
    

In [196]:
cancer_proteins[cancer_proteins['name_x'].isin(ligands_related_to_cancer_proteins)][['name_x', 'class_superclass', 'toxicity', 'patent_approved']].dropna()

,name_x,class_superclass,toxicity,patent_approved
10208,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...
10884,WS821Z52LQ,"6-phenylpteridine-2,4,7-triamine|Teridin|Triam...",Organic compounds,DB00476|DB01235|DB00734|DB01609|DB00022|DB0109...
339356,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...
435539,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...
984620,Enasidenib,Organoheterocyclic compounds,"Based on animal embryo-fetal toxicity studies,...",2016-12-06|2017-08-15|2017-08-22|2018-10-09|20...
...,...,...,...,...
2890356,Ponatinib,Benzenoids,The most common non-hematologic adverse reacti...,2015-05-12|2012-02-14|2016-11-15|2021-12-07|20...
2890360,Ponatinib,Benzenoids,The most common non-hematologic adverse reacti...,2015-05-12|2012-02-14|2016-11-15|2021-12-07|20...
2901554,Dasatinib,Benzenoids,Overdose cases with dasatinib occurred in isol...,2009-08-25|2003-07-22|2006-10-24|2006-12-26|20...
2903139,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...


In [197]:
# Keep only the ligands that are in DrugBank (for now)
drugs_cancer = df[df['name_x'].isin(ligands_related_to_cancer_proteins)].dropna(subset='class_superclass')

In [198]:
len(drugs_cancer)

27997

In [199]:
proteins_related_to_cancer_drugs = drugs_cancer['name_y'].unique()
len(proteins_related_to_cancer_drugs), proteins_related_to_cancer_drugs

(63,
 array([nan, 'CCNA2', 'CCND1', 'CDK6', '72', 'CDK5', 'CDK7', 'CDK9',
        'CDK2', 'PIK3CA', 'gyrB', 'parE', 'PIK3CB', 'PIK3CD', 'PRKAB1',
        'HDAC8', 'TP53', 'MET', 'ALK', 'RET', 'HDAC2', 'HDAC3', 'HSP90AB1',
        'IDH1', 'IL3', 'MAP2K2', 'PRKACB', 'PRKCB', 'MAPK3', 'CAMK2A',
        'PDGFRA', 'HSP90AA1', 'JAK1', 'JAK3', 'JAK2', 'NPM1', 'BCR',
        'ABL2', 'MAPK11', 'AKT2', 'GSK3A', 'CSNK1E', 'EGFR', 'KCNQ1',
        'parC', 'FKBP1A', 'PDE4C', 'FGFR2', 'TOP2B', 'PDE3B', 'RAC1',
        'RAF1', 'ATIC', 'CDK1', 'PIK3CG', 'KDR', 'ESR2', 'EPAS1', 'PRSS2',
        'CACNA1S', 'ROS1', 'TAB1', 'LIMK2'], dtype=object))

In [200]:
df_proteins_related_to_cancer_drugs = df[df['name_y'].isin(proteins_related_to_cancer_drugs)]
len(df_proteins_related_to_cancer_drugs)

2870589

In [201]:
len(set(df_proteins_related_to_cancer_drugs['target_name'].values) - set(cancer_proteins['target_name'].values))

6326

### Using UniProt

In [202]:
cosmic_proteins['Uniprot'].isin(df['swissprot_protein_id']).sum()

np.int64(75)

In [203]:
df['swissprot_protein_id'].isin(cosmic_proteins['Uniprot']).sum()

np.int64(32320)

In [204]:
cancer_proteins = df[df['swissprot_protein_id'].isin(cosmic_proteins['Uniprot'])]

ligands_related_to_cancer_proteins = cancer_proteins['name_x'].unique()
len(ligands_related_to_cancer_proteins), ligands_related_to_cancer_proteins

(240,
 array([nan, 'Alvocidib', 'Olomoucine', 'SNS-032', 'G9ZF61LE7G',
        "N'-(Pyrrolidino[2,1-B]Isoindolin-4-On-8-Yl)-N-(Pyridin-2-Yl)Urea",
        '1-(5-OXO-2,3,5,9B-TETRAHYDRO-1H-PYRROLO[2,1-A]ISOINDOL-9-YL)-3-(5-PYRROLIDIN-2-YL-1H-PYRAZOL-3-YL)-UREA',
        'WS821Z52LQ', 'Pteroic acid', 'K-252a',
        '6,7,12,13-tetrahydro-5H-indolo[2,3-a]pyrrolo[3,4-c]carbazol-5-one',
        'SB-218078', 'Bisindolylmaleimide I', 'Staurosporine',
        '64H5U84UFB',
        '4-[(4-Imidazo[1,2-a]Pyridin-3-Ylpyrimidin-2-Yl)Amino]Benzenesulfonamide',
        'NTZ47MG2A4',
        '4-[4-(4-Methyl-2-Methylamino-Thiazol-5-Yl)-Pyrimidin-2-Ylamino]-Phenol',
        '[4-(2-Amino-4-Methyl-Thiazol-5-Yl)-Pyrimidin-2-Yl]-(3-Nitro-Phenyl)-Amine',
        'Seliciclib',
        "O6-CYCLOHEXYLMETHOXY-2-(4'-SULPHAMOYLANILINO) PURINE",
        'T61871RKRI', 'Wortmannin', '7-Hydroxystaurosporine', 'Pictilisib',
        'TG100-115', 'LY-294002', 'Milciclib Maleate', 'Roginolisib',
        'Tirabrutinib', 

In [205]:
cancer_proteins[cancer_proteins['name_x'].isin(ligands_related_to_cancer_proteins)][['name_x', 'class_superclass', 'toxicity', 'patent_approved']].dropna()

,name_x,class_superclass,toxicity,patent_approved
10208,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...
10884,WS821Z52LQ,"6-phenylpteridine-2,4,7-triamine|Teridin|Triam...",Organic compounds,DB00476|DB01235|DB00734|DB01609|DB00022|DB0109...
339356,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...
339357,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...
435539,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...
...,...,...,...,...
2894015,3X2S926L3Z,TSA,Organic compounds,DB00012|DB00016|DB08894|DB09107|DB00281|DB0029...
2899183,Abemaciclib,Organoheterocyclic compounds,According to the bacterial reverse mutation (A...,2010-12-21
2901554,Dasatinib,Benzenoids,Overdose cases with dasatinib occurred in isol...,2009-08-25|2003-07-22|2006-10-24|2006-12-26|20...
2903139,G9ZF61LE7G,6-acetyl-8-cyclopentyl-5-methyl-2-{[5-(piperaz...,Organic compounds,DB06643|DB00008|DB00011|DB00018|DB00022|DB0003...


In [206]:
# Keep only possible drugs
ligands_cancer = df[df['name_x'].isin(ligands_related_to_cancer_proteins)].dropna(subset='class_superclass')

In [207]:
len(ligands_cancer)

35077

In [208]:
proteins_related_to_cancer_drugs = ligands_cancer['swissprot_protein_id'].unique()
len(proteins_related_to_cancer_drugs), proteins_related_to_cancer_drugs

(104,
 array([<NA>, 'P15206', 'P20248', 'P14635', 'P24864', 'P24385', 'Q00534',
        'Q01043', 'Q00535', 'P50613', 'P50750', 'P24941', 'P42336',
        'P0A0K8', 'P0AES6', 'P20083', 'P42338', 'O00329', 'Q8WYR1',
        'Q9Y478', 'P28222', 'Q9BY41', 'P04637', 'P08581', 'Q9UM73',
        'P07949', 'Q92769', 'Q9UDY8', 'O15379', 'P08238', 'P30281',
        'O75874', 'P30279', 'P08700', 'P36507', 'Q9NYV4', 'P22694',
        'P05771', 'P27361', 'P05696', 'Q9UQM7', 'P16234', 'P07900',
        'Q9Y618', 'P23458', 'P52333', 'O60674', 'P06748', 'P11274',
        'P42684', 'Q15759', 'P31751', 'P26618', 'P49840', 'P49674',
        'P00533', 'P51787', 'P0C1U9', 'P62942', 'Q08493', 'P21802',
        'P33176', 'O62664', 'Q16204', 'Q02880', 'Q13370', 'P20288',
        'P19020', 'Q9NY46', 'P08482', 'P43140', 'P08909', 'P28564',
        'P22086', 'P35355', 'P63000', 'P04049', 'P31939', 'Q15022',
        'P06493', 'O75376', 'O75909', 'Q9P2K3', 'Q9HC35', 'Q09472',
        'P48736', 'P35968', 'Q92731'

In [209]:
df_proteins_related_to_cancer_drugs = df[df['swissprot_protein_id'].isin(proteins_related_to_cancer_drugs)]

In [210]:
secondary_effect_prots = set(df_proteins_related_to_cancer_drugs['target_name'].values) - set(cancer_proteins['target_name'].values)
len(secondary_effect_prots)

6016

In [211]:
df_secondary_effect_prots = df[df['target_name'].isin(secondary_effect_prots)]
df_cancer_prots = df[df['target_name'].isin(cancer_proteins['target_name'])]
df_ligands_cancer = df[df['name_x'].isin(ligands_related_to_cancer_proteins)]

len(df_secondary_effect_prots), len(df_cancer_prots), len(df_ligands_cancer)

(2780623, 67054, 2817554)

In [212]:
len(df_secondary_effect_prots['target_name'].unique()), len(df_cancer_prots['target_name'].unique()), len(df_ligands_cancer['name_x'].unique())

(6016, 142, 240)